In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

import pandas as pd
import numpy as np

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [213]:
feats_import = pd.read_csv('All_Data_2006_2016.csv', index_col=0)

In [228]:
feats = feats_import.drop(['Season', 'Gameday', 'TID_H', 'TID_A', 'HY', 'AY', 'HR', 'AR'], axis=1).drop('FTGD', axis=1)
feats.loc[:, 'HTGD'] = feats.loc[:, 'HTGD'] - feats.loc[:, 'HTGD'].min()
feats = feats/feats.apply(max)

GD_min = int(feats_import.loc[:, 'FTGD'].min())
GD_max = int(feats_import.loc[:, 'FTGD'].max())
label = feats_import.loc[:, 'FTGD'] - GD_min

GD_spread = GD_max-GD_min + 1
ID = np.eye(GD_spread)

X = feats.iloc[:3000].as_matrix()
y_pre = map(int, label.iloc[:3000].as_matrix())

y = np.array([ID[i] for i in y_pre])

In [233]:
def baseline_model():
    model = Sequential()
    model.add(Dense(30, input_dim=len(X[0]), init='uniform', activation='relu')) 
    model.add(Dense(30, activation='sigmoid'))
    model.add(Dense(16, activation='sigmoid'))
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [234]:
estimator = KerasClassifier(build_fn=baseline_model, nb_epoch=100, batch_size=10, verbose=0)

In [235]:
kfold = KFold(n_splits=10, shuffle=True)

In [ ]:
results = cross_val_score(estimator, X, y)#, cv=kfold)
print "Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)

In [194]:
X_test = feats.iloc[2701:].as_matrix()
y_test_pre = map(int, label.iloc[2701:].as_matrix())

y_test = np.array([ID[i] for i in y_test_pre])

In [195]:
oos = cross_val_score(estimator, X_test, y_test, cv=kfold)

In [196]:
print "Baseline: %.2f%% (%.2f%%)" % (oos.mean()*100, oos.std()*100)

Baseline: 21.43% (9.58%)


In [132]:
y_test[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.])

In [136]:
from keras.datasets import mnist

In [139]:
mnist_data = mnist.load_data()